In [16]:
using LinearAlgebra, Statistics, Random, Plots

# 1D Binary Gas Simulation
## December 2025

### Background
Consider a tube of mass M (modeled as two chuncks of mass M/2 at each end of the tube). The tube is filled with a total of N alternating mass particles, $m_1$ and $m_2$ (N/2 of each). The idea behind this simulation is to start out the system in a very non-equilibrium state, and study the approach to equilibrium as the system evolves using elastic collisions. The particle's masses $m_1$ and $m_2$ must be different, since elastic collisions between identical masses merely switch velocities, and then the approach to equilibrium will be completely determined by velocity mixing that occurs when particles 1 and N collide with the ends of the tube

#### Discussion of Physics and incremental code build

In one dimension, conservation of kinetic energy and momentum lead to the collision rule
$$\begin{pmatrix} v_1' \\ v_2' \end{pmatrix} = 
\frac{1}{m_1 + m_2}
\begin{pmatrix} 
m_1 - m_2 & 2m_2 \\ 
2m_1 & m_2 - m_1 
\end{pmatrix}
\begin{pmatrix} v_1 \\ v_2 \end{pmatrix}
$$


In [17]:
    """
    function collide(m1,m2, v1,v2)
    This routine performs the collision physics assuming perfectly elastic 
    collisions. 
    """
function collide(m1,m2,v1,v2)
    v1New = ((m1-m2)*v1 + 2*m2*v2) / (m1+m2)
    v2New = (2*m1*v1 + (m2-m1)*v2) / (m1+m2)
    return v1New, v2New
end

# Handle simultaneous (or near-simultaneous) adjacent collisions
function handleNextCollisions!(particles; R=R, tol=1e-12)
    n = length(particles)
    Δt_ij = fill(Inf, n-1)
    for i in 1:n-1
        if particles[i].v > particles[i+1].v
            Δt_ij[i] = ((particles[i+1].x - particles[i].x) - 2R) / (particles[i].v - particles[i+1].v)
        end
    end
    tmin = minimum(Δt_ij)
    if !isfinite(tmin)
        return Inf
    end
    idxs = findall(i -> abs(Δt_ij[i] - tmin) <= tol, 1:length(Δt_ij))

    # advance all particle positions by tmin (using pre-collision velocities)
    for p in particles
        p.x += p.v * tmin
    end

    # group contiguous indices into connected components (chains)
    function group_contiguous(sorted_idxs)
        groups = []
        if isempty(sorted_idxs)
            return groups
        end
        current = [sorted_idxs[1]]
        for k in 2:length(sorted_idxs)
            if sorted_idxs[k] == sorted_idxs[k-1] + 1
                push!(current, sorted_idxs[k])
            else
                push!(groups, copy(current))
                empty!(current)
                push!(current, sorted_idxs[k])
            end
        end
        push!(groups, copy(current))
        return groups
    end

    groups = group_contiguous(sort(idxs))

    # resolve each group
    for g in groups
        if length(g) == 1
            i = g[1]
            v1 = particles[i].v
            v2 = particles[i+1].v
            m1 = particles[i].m
            m2 = particles[i+1].m
            v1new, v2new = collide(m1, m2, v1, v2)
            particles[i].v   = v1new
            particles[i+1].v = v2new
        else
            a = g[1]
            c = g[end]
            b = c + 1
            v = [particles[j].v for j in a:b]
            m = [particles[j].m for j in a:b]
            nblock = length(v)

            changed = true
            max_iter = 10 * nblock
            iter = 0
            while changed && (iter < max_iter)
                changed = false
                iter += 1
                for local_i in 1:(nblock-1)
                    if v[local_i] > v[local_i+1]
                        v[local_i], v[local_i+1] = collide(m[local_i], m[local_i+1], v[local_i], v[local_i+1])
                        changed = true
                    end
                end
            end
            if iter >= max_iter
                @warn "Iterative collision resolution hit iteration cap for chain [$a:$b]."
            end
            for offset in 0:(nblock-1)
                particles[a + offset].v = v[offset+1]
            end
        end
    end

    return tmin
end

handleNextCollisions! (generic function with 1 method)

### Define the simulation parameters

In [15]:
M  = 1.000   # all masses in kg
m₁ = 0.100 
m₂ = 0.200
v₁ = 0.010   # m/s
v₂ = 0.020
Nₚ =  5       # number of pairs of particles
N  =  2*Nₚ;    # total number of particles
R  =  0.0005;  # radius of each particle in m

Now use the above info to determine a minimum length for the tube. Eventually, I will want to place all N particles on the left half of the tube. so use this info along with the radius of each particle to set the minimum length $L_{\mathrm{min}} > 4R(N+1)$:

In [18]:
Lmin = 4R*(N+1)
L = 1.000 # in meters
if ( L<Lmin)
    println("The value you chose for L is too small! Choosing L = 10*Lmin") 
    L = 10*Lmin
end
Δx = L/(2*(N+1))

0.045454545454545456

Now create a struct for each particle: 

In [19]:
# Mutable version (allows changing values)
mutable struct Particle
     r::Float64    # radius
    m::Float64  # mass
    x::Float64  # position
    v::Float64  # velocity
end

In [20]:
function fillArray(M, m₁, m₂, v₁,v₂, N, R)
    particles = []
    for i in 1:N+2
        if i==1
            push!(particles,  Particle(R, M/2.0,  0.0, 0.0 ) ) # left end of tube (initially at rest)
        elseif i == N+2
            push!(particles, Particle(R, M/2.0, L, 0.0 ))   # right end of tube (initially at rest)
        else
            if iseven(i)
                push!(particles, Particle(R, m₂, (i-1)*Δx, v₂*(1.0 + 0.01*rand()) ) )
            else 
                push!(particles, Particle(R, m₁, (i-1)*Δx, v₁*(1.0 + 0.01*rand()) ) )
            end
        end
    end  
    return particles
end

fillArray (generic function with 1 method)

In [21]:
particles = fillArray(M, m₁, m₂, v₁,v₂, N, R);

In [22]:
function findNextCollision(particles)
    Δt_ij = zeros(length(particles)-1)
    for i in 1:length(particles)-1
        if particles[i].v > particles[i+1].v
            Δt_ij[i] = ( (particles[i+1].x - particles[i].x) - 2R ) / (particles[i].v - particles[i+1].v)
        else
            Δt_ij[i] = Inf
        end
    end
    tNext, idx = findmin(Δt_ij)
    return tNext, idx
end

findNextCollision (generic function with 1 method)

In [13]:
tNext, idx = findNextCollision(particles)

(4.394127739648634, 2)

#### Test: Three-particle chain collision

This demo tests the simultaneous collision handler with a controlled scenario: three particles arranged so that two adjacent collision pairs occur at nearly the same time (a chain collision).


In [23]:
# Demo: Three-particle chain collision
# Set up a simple scenario with three particles where two collisions happen simultaneously
test_particles = [
    Particle(R, 1.0, 0.0, 0.0),      # particle 0: fixed left wall
    Particle(R, m₁, 0.001, 1.0),     # particle 1: moving right
    Particle(R, m₂, 0.002, 0.5),     # particle 2: slower, moving right
    Particle(R, m₁, 0.003, 0.0),     # particle 3: at rest
]

println("=== Before collision ===")
for (i, p) in enumerate(test_particles)
    println("Particle $i: x=$(round(p.x, digits=4)), v=$(round(p.v, digits=4))")
end

# Apply the collision handler
tCollide = handleNextCollisions!(test_particles; R=R, tol=1e-12)
println("\nCollision time: $tCollide")

println("\n=== After collision ===")
for (i, p) in enumerate(test_particles)
    println("Particle $i: x=$(round(p.x, digits=4)), v=$(round(p.v, digits=4))")
end

# Verify momentum conservation (approximately)
total_momentum_before = sum([p.m for p in test_particles] .* [0.0, 1.0, 0.5, 0.0])  # pre-collision velocities
total_momentum_after = sum([p.m for p in test_particles] .* [p.v for p in test_particles])
println("\nMomentum before: $(round(total_momentum_before, digits=6))")
println("Momentum after:  $(round(total_momentum_after, digits=6))")


=== Before collision ===
Particle 1: x=0.0, v=0.0
Particle 2: x=0.001, v=1.0
Particle 3: x=0.002, v=0.5
Particle 4: x=0.003, v=0.0

Collision time: 0.0
Particle 1: x=0.0, v=0.0
Particle 2: x=0.001, v=1.0
Particle 3: x=0.002, v=0.5
Particle 4: x=0.003, v=0.0

Collision time: 0.0

=== After collision ===
Particle 1: x=0.0, v=0.0
Particle 2: x=0.001, v=0.2593
Particle 3: x=0.002, v=0.3148
Particle 4: x=0.003, v=1.1111

Momentum before: 0.2
Momentum after:  0.2

=== After collision ===
Particle 1: x=0.0, v=0.0
Particle 2: x=0.001, v=0.2593
Particle 3: x=0.002, v=0.3148
Particle 4: x=0.003, v=1.1111

Momentum before: 0.2
Momentum after:  0.2
